In [22]:
import pandas as pd
import numpy as np
import random as rand
import importlib

In [11]:
raw = pd.read_csv("yellow-taxis/1january.csv")

In [12]:
# delete unused columns
# del raw['trip_distance']
del raw['passenger_count']
del raw['fare_amount']
del raw['tolls_amount']
del raw['taxes_amount']
del raw['tip_amount']
# del raw['payment_amount']
del raw['payment_type']

In [13]:
# convert pickup_dtatetime to datetime
raw["pickup_datetime"] = pd.to_datetime(raw["pickup_datetime"])
raw["dropoff_datetime"] = pd.to_datetime(raw["dropoff_datetime"])

In [14]:
#make sure types are okay
print(list(raw.columns.values))
print([raw[i].dtype for i in list(raw.columns.values)])

['pickup_datetime', 'pickup_latitude', 'pickup_longitude', 'trip_distance', 'dropoff_datetime', 'dropoff_latitude', 'dropoff_longitude', 'payment_amount']
[dtype('<M8[ns]'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('<M8[ns]'), dtype('float64'), dtype('float64'), dtype('float64')]


In [15]:
#convert it to a numpy matrix
#np_raw = raw.as_matrix()

In [16]:
#print(np_raw.dtype)
print(raw.ix[0])
print(len(raw))

pickup_datetime      2016-01-01 00:00:00
pickup_latitude                  40.7347
pickup_longitude                -73.9904
trip_distance                        1.1
dropoff_datetime     2016-01-01 00:00:00
dropoff_latitude                 40.7324
dropoff_longitude               -73.9818
payment_amount                       8.8
Name: 0, dtype: object
10906858


In [73]:
import State as State
importlib.reload(State)        

class MarkovChain:
    # num centers are we picking for k-means
    def __init__(self, raw, k):
        self.state_set = set()
        self.id_to_state = {}
        self.adj_matrix = None
        self.raw = raw
        
        self.initialize_centers(k)
        epsilon = 1
        self.build_states_kmeans(1001, epsilon)
        
        self.add_points_edges()
        self.make_adjacency_matrix()
    
    def initialize_centers(self, k):
        ind = [i for i in range(len(self.raw))]
        rand.shuffle(ind)
        centers = ind[:k]
        # initialize centers
        ident = 0
        for c_ind in centers:
            # out of convenience, we aren't messing with pickup lat lon
            lat = self.raw.ix[c_ind]["dropoff_latitude"]
            lon = self.raw.ix[c_ind]["dropoff_longitude"]
            s = State.State((lat, lon), ident)
            self.state_set.add(s)
            self.id_to_state[ident] = s
            ident += 1
    
    def build_states_kmeans(self, iterations, epsilon):
        # run kmeans algorithm
        min_diff = 1000000
        while iterations > 0 and min_diff > epsilon:
            for ind, row in self.raw.iterrows():
                pos_start, pos_end = self.row_to_positions(row)
                closest_to_start = self.find_closest_state(pos_start)
                closest_to_end = self.find_closest_state(pos_end)
                
                closest_to_start.add_position(pos_start)
                closest_to_end.add_position(pos_end)
            max_diff = 0
            for s in self.state_set:
                max_diff = max(max_diff, s.update_center())
            min_diff = min(min_diff, max_diff)
            iterations -= 1
        print(iterations) 

    def add_points_edges(self):
        for s in self.state_set:
            s.clear_stored_data()
        for ind, row in self.raw.iterrows():
            pos_start, pos_end = self.row_to_positions(row)
            closest_to_start = self.find_closest_state(pos_start)
            closest_to_end = self.find_closest_state(pos_end)
            
            fare = self.row_to_fare(row)
            tdistance = self.row_to_distance(row)
            
            #Add points to respective states
            closest_to_start.store_data((pos_start, fare, tdistance))
            closest_to_end.store_data((pos_end))
            
            ##Add this edge to markov state
            closest_to_start.add_destination(closest_to_end.id)
    
    
    def make_adjacency_matrix(self):
        self.adj_matrix = np.ndarray(shape=(len(self.state_set), len(self.state_set)), dtype=float, order='C')
        for i in sorted(self.id_to_state.keys()):
            for j in sorted(self.id_to_state.keys()):
                self.adj_matrix[i][j] = self.transition_probability(i, j)
    
    def sum_of_square_error(self):
        total = 0
        for s in self.state_set:
            total += s.sum_of_squared_errors
        return total
    
    ##
    # GETTERS
    ##
    def get_adjacency_matrix(self):
        return self.adj_matrix
    
    ###
    # HELPER METHODS
    ###
    def find_closest_state(self, pos):
        def distance(state, pos):
            clat, clon = state.center
            return ((clat - pos[0])**2 + (clon - pos[1])**2)**0.5
        closest = None
        min_dist = None
        for state in self.state_set:
            d = distance(state, pos)
            if closest == None or d < min_dist:
                closest = state
                min_dist = d
        return closest
    
    def row_to_positions(self, row):
        lats = row["pickup_latitude"]
        lons = row["pickup_longitude"]
        pos_start = (lats, lons)

        late = row["dropoff_latitude"]
        lone = row["dropoff_longitude"]
        pos_end = (late, lone)
        
        return pos_start, pos_end
    
    def row_to_fare(self, row):
        return row["payment_amount"]
    def row_to_distance(self, row):
        return row["trip_distance"]
    
    def transition_probability(self, i, j):
        return self.id_to_state[i].probability_to(j)


In [74]:
def find_optimal_k(raw, k_list):
    points_to_plot = []
    for k in k_list:
        m = MarkovChain(raw, k)
        points_to_plot.append((k, m.sum_of_square_error))
    return points_to_plot

In [75]:
m = MarkovChain(raw[:1000], 10)

TypeError: unorderable types: tuple() > int()

In [41]:
adjm = m.get_adjacency_matrix()

In [42]:
np.sum(adjm, axis=1)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [43]:
S, U = np.linalg.eig(adjm.T)

In [44]:
print(U.T)

[[ 0.19358393+0.j          0.09721960+0.j          0.10873574+0.j
   0.23783651+0.j          0.51046406+0.j          0.25715077+0.j
   0.36466965+0.j          0.16214579+0.j          0.59883215+0.j
   0.20027161+0.j        ]
 [ 0.13056937+0.j          0.07500944+0.j          0.01773779+0.j
  -0.83612310+0.j         -0.18989318+0.j          0.17222964+0.j
   0.09093622+0.j          0.11229875+0.j          0.43727015+0.j
  -0.01003506+0.j        ]
 [-0.07820002+0.j         -0.03715973+0.j         -0.09539010+0.j
   0.55471087+0.j          0.36971653+0.j         -0.32384248+0.j
   0.35620427+0.j         -0.08828439+0.j         -0.53276448+0.j
  -0.12499047+0.j        ]
 [ 0.17303157+0.j         -0.03121150+0.j         -0.05109706+0.j
  -0.06396154+0.j         -0.80404449+0.j          0.17619538+0.j
   0.50978139+0.j          0.05021589+0.j         -0.08339871+0.j
   0.12448908+0.j        ]
 [ 0.17197345+0.j          0.25910795+0.j         -0.12946616+0.j
   0.12504652+0.j         -0.77671

In [45]:
# U is row major
inv_dist = U.T[0]
print(inv_dist)

[ 0.19358393+0.j  0.09721960+0.j  0.10873574+0.j  0.23783651+0.j
  0.51046406+0.j  0.25715077+0.j  0.36466965+0.j  0.16214579+0.j
  0.59883215+0.j  0.20027161+0.j]


In [46]:
norm_inv_dist = inv_dist / float(sum(inv_dist))

/Users/cheng/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  if __name__ == '__main__':


In [47]:
inv2 = sorted(norm_inv_dist)

In [48]:
old_norm_inv_dist = sorted(norm_inv_dist)

In [49]:
old_norm_inv_dist

[(0.035599711401181802+0j),
 (0.039816672398490988+0j),
 (0.059374273692836953+0j),
 (0.070886241315251799+0j),
 (0.073335123072190422+0j),
 (0.087090577001469799+0j),
 (0.094163040739149148+0j),
 (0.13353412332344694+0j),
 (0.18692087726919729+0j),
 (0.21927935978678498+0j)]